In [14]:
import torch
import torch.nn as nn
from torchsummary import summary
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.resnet import Bottleneck

In [6]:
class ResBlock(nn.Module):
    def __init__(self, in_planes, planes, stride):
        super(ResBlock, self).__init__()
        self.flag = False

        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(in_planes, in_planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
                
        self.conv3 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)

        if stride != 1 or in_planes != planes:
            self.flag = True
        self.shortcut_conv = nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False)
        self.shortcut_bn = nn.BatchNorm2d(planes)

        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        if self.flag:
          x = self.shortcut_bn(self.shortcut_conv(x))
        out += x
        out = self.relu(out)

        return out

In [7]:
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.conv0 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn0 = nn.BatchNorm2d(64)
        self.pooling = nn.MaxPool2d(3,2)
        self.relu = torch.nn.LeakyReLU(inplace=True)
        
        self.resB_11 = ResBlock(64,256,2)
        self.resB_11 = ResBlock(64,256,2)
        self.resB_11 = ResBlock(64,256,2)
    def forward(self, x):
        x = self.bn0(self.conv0(x))
        x = self.pooling(self.relu(x))
        return x
    
    
resnet = ResNet50().cuda()
summary(resnet,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
         LeakyReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 55, 55]               0
Total params: 9,600
Trainable params: 9,600
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 19.85
Params size (MB): 0.04
Estimated Total Size (MB): 20.46
----------------------------------------------------------------


In [39]:
resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1).cuda()
btt = Bottleneck(64,64).cuda()
summary(btt,(64,56,56))

RuntimeError: The size of tensor a (256) must match the size of tensor b (64) at non-singleton dimension 1